In [1]:
using Pkg; Pkg.activate("..")
using Revise
using Geom4hep

  Activating environment at `~/Development/VecGeom/Geom4hep/Project.toml`


In [10]:
# Geometry construction
world = Volume("world", Box(100.,100.,100.), Material("vacuum"))
box = Volume("box", Box(10.,10.,10.), Material("iron"))
placeDaughter!(world, Transformation3D(), box)

1-element Vector{Geom4hep.AbstractPlacedVolume{Float64}}:
 Geom4hep.PlacedVolume{Float64}(Transformation3D{Float64}([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0]), Volume{Float64}("box", Box{Float64}([10.0, 10.0, 10.0]), Material{Float64}("iron"), Geom4hep.AbstractPlacedVolume{Float64}[]))

In [22]:
using GLMakie
function draw(vol::Volume) 
    scene = mesh(toMesh(vol.shape), color=:grey, transparency=true, ambient=0.7)
    for daughter in vol.daughters
        mesh!(toMesh(daughter.volume.shape), color=:blue, transparency=true, ambient=0.7)
    end
    display(scene)
end
draw(world)